# ESPM / IB 105
The goal of this notebook is to access and integrate diverse data sets to visualize correlations and discover patterns to address questions of species’ responses to environmental change. We will use programmatic tools to show how to use Berkeley resources such as the biodiversity data from biocollections and online databases, field stations, climate models, and other environmental data.

Run this code first. This imports all the libraries, which are sets of reusable functions and resources that someone else wrote. Since writing all the code for sending requests to a server is out of the scope of this class, we need to use a library that someone else built to do that for us! You can google each library to see what its purpose is. 

Another term you will see is API (application programming interface), which refers to the functions/methods in a library that you can call to ask it to do things for you. 

In [ ]:
%matplotlib inline
import os
from datetime import datetime

from descartes import PolygonPatch
import matplotlib as mpl
from matplotlib.collections import PatchCollection
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
from shapely import geometry as sg, wkt

In this example we will demonstrate the intersection of collection data, field station boundaries and future climate datasets. Essentially, we will be parsing a lot of data and trying to display it in a meaningful manner. 

Choose a matplotlib style. See `plt.style.available` for all the available styles. For now, we will use Seaborn. 

In [ ]:
plt.style.use('seaborn')

The [GBIF Web API](http://www.gbif.org/developer/summary) utilizes pagination to split up the returned JSON records, so we need to request each successive page to gather all the results. This helper class will handle that.

In [ ]:
class GBIFRequest(object):
    """GBIF requests with pagination handling."""
    url = 'http://api.gbif.org/v1/occurrence/search'

    def fetch(self, params):
        resp = requests.get(self.url, params=params)
        return resp.json()

    def get_pages(self, params):
        params = dict({'limit': 100, 'offset': 0}, **params)
        data = {'endOfRecords': False}
        while not data['endOfRecords']:
            data = self.fetch(params)
            params['offset'] += params['limit']
            yield data

Then, we can request the available occurrence records for a particular species and plot their locations. Here we're going to make a request for the species whose scientific name is 'Argia agrioides', also known as the [California dancer](https://www.google.com/search?q=Argia+agrioides&rlz=1C1CHBF_enUS734US734&source=lnms&tbm=isch&sa=X&ved=0ahUKEwji9t29kNTWAhVBymMKHWJ-ANcQ_AUICygC&biw=1536&bih=694). We're going to plot their locations and display other data results. 

In [ ]:
req = GBIFRequest()
params = {'scientificName': 'Argia agrioides'}
pages = req.get_pages(params)
# Filter all the returned records for valid locations.
records = [rec for page in pages for rec in page['results']
           if rec.get('decimalLatitude')]
# Make a list of coordinate pairs for plotting.
coords = [(r['decimalLongitude'], r['decimalLatitude']) for r in records]
xs, ys = zip(*coords)

In [ ]:
# Make a plot of species' locations below.
plt.plot(xs, ys, 'ro', alpha=0.5)

For all the reserves, we will need to send a request to GeoJSON, which is a format for encoding a variety of geographic data structures. With this code, we can request GeoJSON for all reserves and plot ocurrences of the species.

In [ ]:
# This is the url where are data is located 
url = 'https://ecoengine.berkeley.edu/api/layers/reserves/features/'
# Make a request, which receives data from this url. Then iterate throuh each feature so we can collect data about them.
reserves = requests.get(url, params={'page_size': 30}).json()
geoms = {feat['properties']['name']: sg.asShape(feat['geometry']) for feat in reserves['features']}

In [ ]:
# Buffer the reserves so we can view them at a regional scale.
patches = [PolygonPatch(geom.buffer(0.25)) for geom in geoms.values()]
fig, ax = plt.subplots()
colors = 100 * np.random.rand(len(patches))
p = PatchCollection(patches, cmap='tab10', alpha=0.4)
p.set_array(np.array(colors))
ax.add_collection(p)
ax.autoscale()

Let's try and zoom in to the Santa Cruz Island Reserve where we have a single occurrence, and plot it similarly. 

In [ ]:
# Collect data about what we want to graph.
fig, ax = plt.subplots()
g = geoms['Santa Cruz Island Reserve']
poly = PolygonPatch(g)
ax.add_patch(poly)
# Plot a graph representing the Santa Cruz Island Reserve using the data we've accumulated in these variables.
ax.plot(xs, ys, 'ro', clip_path=poly)
xmin, ymin, xmax, ymax = g.bounds
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)

In the same way, we can parse the WKT geometry for Blodgett, Hopland, and Sagehen stations and plot them individually.

In [ ]:
station_urls = {
    'Blodgett': 'https://raw.githubusercontent.com/BNHM/spatial-layers/master/wkt/BlodgettForestResearchStation.wkt',
    'Hopland': 'https://raw.githubusercontent.com/BNHM/spatial-layers/master/wkt/HoplandREC.wkt',
    'Sagehen': 'https://raw.githubusercontent.com/BNHM/spatial-layers/master/wkt/SagehenCreekFieldStation.wkt'
}
stn_features = [{'id': name, 'geometry': wkt.loads(requests.get(url).text)}
                for name, url in station_urls.items()]
# Reduce geometric complexity, we need smaller geometry serializations to circumvent 
# URL length restrictions.
tol = .0001
for feat in stn_features:
    feat['geometry'] = feat['geometry'].buffer(tol).simplify(tol)

In [ ]:
f, axes = plt.subplots(1, len(stn_features))
f.set_size_inches((12, 6))

# Title our graphs.
for ax, feat in zip(axes, stn_features):
    ax.add_patch(PolygonPatch(feat['geometry']))
    ax.set_title(feat['id'])
    ax.autoscale()

This will use the [Cal-Adapt](http://api.cal-adapt.org/api/) web API to access a remote raster data source and preview a rendered image. In this case, the data source contains images of locations and temperature information corresponding to each one. 

In [ ]:
url = 'http://api.cal-adapt.org/api/series/tasmax_year_CanESM2_rcp85/rasters/'
json = requests.get(url).json()
tifurl = json['results'][0]['image']
title = json['results'][0]['slug']

In [ ]:
import greenwich

In [ ]:
with greenwich.open(tifurl) as r:
    arr = r.masked_array()

Now we need to visualize the resulting masked array. The array values are maximum temperature in Kelvin.

In [ ]:
# Use Matplotlib to create a color-coded plot of the resulting masked array:
plt.imshow(arr, cmap='spectral')
plt.title(title)
plt.colorbar()

In [ ]:
# Use Matplotlib to plot a histogram (arr should be un-masked here)

h = plt.hist(arr.compressed(), 20)

As we look closer at the Cal-Adapt API, the following will ease working with time series raster data. It will request an entire time series for any geometry and return a Pandas `DataFrame` object. This will make displaying the data much easier with Pandas' built in methods. 

In [ ]:
def to_fahren(val):
    return (val - 273.15) * 9 / 5 + 32


class CalAdaptRequest(object):
    series_url = 'http://api.cal-adapt.org/api/series/'

    def __init__(self, slug=None):
        self.slug = slug or 'tasmax_year_CanESM2_rcp85'
        self.params = {'pagesize': 94}
    
    def concat_features(self, features, field='id'):
        lst = []
        for feat in features:
            json = self.series(geom=feat['geometry'])
            series = self.to_frame(json)['image']
            if series.any():
                series.name = feat[field]
                lst.append(series)
        return pd.concat(lst, axis=1)
    
    def list_series_slugs(self):
        json = requests.get(self.series_url, params=self.params).json()
        return [row['slug'] for row in json['results']]

    def series(self, geom=None):
        url = '%s%s/rasters/' % (self.series_url, self.slug)
        if geom:
            params = dict(self.params, g=geom.wkt)
            if isinstance(geom, (sg.Polygon, sg.MultiPolygon)):
                params['stat'] = 'mean'
        return requests.get(url, params=params).json()
    
    def series(self, geom=None, dates=None):
        if dates:
            url = os.path.join(self.series_url, self.slug, *dates)
        else:
            url = os.path.join(self.series_url, self.slug, 'rasters/')
        if geom:
            params = dict(self.params, g=geom.wkt)
            if isinstance(geom, (sg.Polygon, sg.MultiPolygon)):
                params['stat'] = 'mean'
        return requests.get(url, params=params).json()
    
    def to_frame(self, json):
        df = pd.DataFrame.from_records(json['results'])
        df['image'] = to_fahren(pd.to_numeric(df['image']))
        df['event'] = pd.to_datetime(df['event'], format='%Y-%m-%d')
        df.set_index('event', inplace=True)
        return df.dropna()

In [ ]:
# Creating Pandas dataframe by combining requested time series with records_g
req = CalAdaptRequest()
records_g = [dict(rec, geometry=sg.Point(rec['decimalLongitude'], rec['decimalLatitude']))
             for rec in records]
df = req.concat_features(records_g, 'gbifID')

In [ ]:
df.iloc[:,:9].plot()
plt.title('Argia agrioides - %s' % req.slug)

We can map the projected means for occurrence locations.

In [ ]:
tmax_means = df.mean(axis=0)
# Filter GBIF records where we have climate data.
records_rvals = [rec for rec in records_g if rec['gbifID'] in tmax_means.index]
coords = [(r['decimalLongitude'], r['decimalLatitude']) for r in records_rvals]
xs2, ys2 = zip(*coords)
#plt.scatter(xs2, ys2, c=tmax_means, cmap='viridis', alpha=0.5)
norm = mpl.colors.Normalize()
size = np.pi * (15 * norm(tmax_means)) ** 2 
plt.scatter(xs2, ys2, c=tmax_means, s=size, cmap='viridis', alpha=0.5)
plt.colorbar()

In [ ]:
#plt.figure(figsize=(12, 6))
df.iloc[:,:7].plot.box()
plt.title('Argia agrioides - %s' % req.slug)

Taking a look at minimum temperatures now, here are the temperature distributions by decade.

In [ ]:
dfs = []
names = ('tasmin_year_CanESM2_rcp45', 'tasmin_year_CanESM2_rcp85')
for name in names:
    req = CalAdaptRequest(name)
    dfs.append(req.concat_features(records_g, 'gbifID'))

In [ ]:
for name, df in zip(names, dfs):
    t = df.resample('10A').mean().transpose()
    t.columns = t.columns.year
    t.plot.box()
    plt.title(name)

Make plots for historical and projected temperatures at three station locations. Once again, Pandas gives us a lot of functionality in displaying our data beautifully. 

In [ ]:
dfstns_lst = []
names = ('tasmax_year_CanESM2_historical', 'tasmax_year_CanESM2_rcp85')
# Iterate through each name and make requests using the CalAdapt API. We can also use Pandas to display this data efficiently.
for name in names:
    req = CalAdaptRequest(name)
    d = req.concat_features(stn_features)
    dfstns_lst.append(d)
    #d.plot()
    #plt.title(name)
dfstns = pd.concat(dfstns_lst)

In [ ]:
dfstns.describe()

In [ ]:
dfstns.plot()

In [ ]:
# Decadal average and difference from 20 years prior.
dfstns.resample('10A').mean().diff(periods=2).dropna()

In [ ]:
dfstns.resample('10A').mean().diff().plot()